In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
log_name = 'test'
with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)

/tmp/ipykernel_892906/593058375.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/pcr/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A, {},
                                     test_event_log, n=N)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                                                                            | 0/1235 [00:00<?, ?it/s]Process ForkPoolWorker-89:
Process ForkPoolWorker-3:
  0%|                                                                                                            | 0/1235 [00:02<?, ?it/s]Process ForkPoolWorker-69:
Process ForkPoolWorker-61:
Process ForkPoolWorker-91:
Process ForkPoolWorker-35:
Process ForkPoolWorker-38:
Process ForkPoolWorker-87:
Process ForkPoolWorker-78:
Process ForkPoolWorker-90:

Process ForkPoolWorker-41:
Process ForkPoolWorker-28:
Process ForkPoolWorker-40:
Process ForkPoolWorker-55:
Process ForkPoolWorker-84:
Process ForkPoolWorker-93:
Process ForkPoolWorker-94:
Process ForkPoolWorker-65:
Process ForkPoolWorker-92:
Process ForkPoolWorker-74:
Process ForkPoolWorker-19:
Process ForkPoolWorker-86:
Process ForkPoolWorker-42:
Process ForkPoolWorker-77:
Process ForkPoolWorker-68:
Process ForkPoolWorker-85:
Process ForkPoolWorker

KeyboardInterrupt: 

In [ ]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A))

In [ ]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S, {},
                                     test_event_log, n=N)
likelihoods_A_S = evaluator_A_S.sample_cases(False, False)

In [ ]:
np.mean([v.ln() for v in likelihoods_A_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S))

In [ ]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D, {},
                                     test_event_log, n=N)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False)

In [ ]:
np.mean([v.ln() for v in likelihoods_A_S_D[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S_D))

In [ ]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC, {},
                                     test_event_log, n=N)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False)

In [ ]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [ ]:
np.mean([v.ln() for v in likelihoods_A_S_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S_AC))

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)